# Week 42
Group: Eyyüb Güven, Alicja Terelak, Giorgio
Chirio

In [2]:
from random import random, seed
import numpy as np
import autograd.numpy as np
import matplotlib.pyplot as plt
from autograd import grad

def CostOLS(beta):
    return (1.0/n)*np.sum((y-X @ beta)**2)




n=100
x = 2*np.random.rand(n,1)
y = 2+(4*x)+(5*(x**2))


X = np.c_[np.ones((n,1)),x, x**2]

X_TX = X.T @ X

linreg_theta = np.linalg.inv(X.T @ X) @ (X.T @ y)
print("our own inversion:")
print(linreg_theta)

# PLAIN GD

H = (2/n) * X_TX
EigVal, EigVect = np.linalg.eig(H)

theta = np.random.randn(3,1)
eta = 1.0/np.max(EigVal)
Niterations = 1000
training_gradient = grad(CostOLS)

# Base Gradient Descent
for interation in range(Niterations):
    grads = training_gradient(theta)
    theta -= eta*grads


print("theta from our own grad descent:")
print(theta)


# GD with AdaGrad, plain GD without momentum


theta = np.random.randn(3,1)
eta = 0.1
Niterations = 1000
training_gradient = grad(CostOLS)
Giter = 0.0
delta = 1e-8
# Base Gradient Descent
for interation in range(Niterations):
    grads = training_gradient(theta)
    Giter += grads*grads
    update = grads*eta/(delta+np.sqrt(Giter))
    theta -= update

print("theta from our own grad descent, using adagrad:")
print(theta)

# GD with AdaGrad, plain GD with momentum


theta = np.random.randn(3,1)
eta = 1.0/np.max(EigVal)
Niterations = 1000
training_gradient = grad(CostOLS)
Giter = 0
delta = 1e-8
momentum = 0.3
update = 0
# Base Gradient Descent
for interation in range(Niterations):
    grads = training_gradient(theta)
    Giter += grads*grads
    new_update = grads*eta/(delta+np.sqrt(Giter)) + momentum * update
    theta -= new_update
    update = new_update

print("theta from our own grad descent, using adagrad and momentum of 0.3:")
print(theta)



# SGD with AdaGrad, without momentum

def NewCostOLS(y,X,theta):
    return np.sum((y-X @ theta)**2)


training_gradient = grad(NewCostOLS,2)
# Define parameters for Stochastic Gradient Descent
n_epochs = 50
M = 5   #size of each minibatch
m = int(n/M) #number of minibatches
# Guess for unknown parameters theta
theta = np.random.randn(3,1)
# Value for learning rate
eta = 1.0/np.max(EigVal)
# Including AdaGrad parameter to avoid possible division by zero
delta  = 1e-8
for epoch in range(n_epochs):
    Giter = 0.0
    for i in range(m):
        random_index = M*np.random.randint(m)
        xi = X[random_index:random_index+M]
        yi = y[random_index:random_index+M]
        gradients = (1.0/M)*training_gradient(yi, xi, theta)
        Giter += gradients*gradients
        update = gradients*eta/(delta+np.sqrt(Giter))
        theta -= update
print("theta from SGD with AdaGrad and without momentum")
print(theta)

# SGD with AdaGrad, with momentum

training_gradient = grad(NewCostOLS,2)
# Define parameters for Stochastic Gradient Descent
n_epochs = 50
M = 5   #size of each minibatch
m = int(n/M) #number of minibatches
# Guess for unknown parameters theta
theta = np.random.randn(3,1)
# Value for learning rate
eta = 1.0/np.max(EigVal)
# Including AdaGrad parameter to avoid possible division by zero
delta  = 1e-8
momentum = 0.3
old_update = 0
for epoch in range(n_epochs):
    Giter = 0.0
    for i in range(m):
        random_index = M*np.random.randint(m)
        xi = X[random_index:random_index+M]
        yi = y[random_index:random_index+M]
        gradients = (1.0/M)*training_gradient(yi, xi, theta)
        Giter += gradients*gradients
        update = gradients*eta/(delta+np.sqrt(Giter)) + momentum * old_update
        theta -= update
        old_update = update
print("theta from SGD with AdaGrad and with momentum")
print(theta)

# RMS prop

training_gradient = grad(NewCostOLS,2)
# Define parameters for Stochastic Gradient Descent
n_epochs = 50
M = 5   #size of each minibatch
m = int(n/M) #number of minibatches
# Guess for unknown parameters theta
theta = np.random.randn(3,1)
# Value for learning rate
eta = 1.0/np.max(EigVal)

rho = 0.99
# Including AdaGrad parameter to avoid possible division by zero
delta  = 1e-8
for epoch in range(n_epochs):
    Giter = 0.0
    for i in range(m):
        random_index = M*np.random.randint(m)
        xi = X[random_index:random_index+M]
        yi = y[random_index:random_index+M]
        gradients = (1.0/M)*training_gradient(yi, xi, theta)
        Giter += (rho*Giter+(1-rho)*gradients*gradients)
        update = gradients*eta/(delta+np.sqrt(Giter))
        theta -= update
print("theta from RMSprop using autograd")
print(theta)

# ADAM
training_gradient = grad(NewCostOLS,2)
# Define parameters for Stochastic Gradient Descent
n_epochs = 50
M = 5   #size of each minibatch
m = int(n/M) #number of minibatches
# Guess for unknown parameters theta
theta = np.random.randn(3,1)
# Value for learning rate
eta = 1.0/np.max(EigVal)
beta1 = 0.9
beta2 = 0.999
# Including AdaGrad parameter to avoid possible division by zero
delta  = 1e-8
iter = 0
for epoch in range(n_epochs):
    mom_1 = 0.0
    mom_2 = 0.0
    iter += 1
    for i in range(m):
        random_index = M*np.random.randint(m)
        xi = X[random_index:random_index+M]
        yi = y[random_index:random_index+M]
        gradients = (1.0/M)*training_gradient(yi, xi, theta)
        mom_1 = beta1*mom_1 + (1-beta1)*gradients
        mom_2 = beta2*mom_2 + (1-beta2)*gradients*gradients
        term_1 = mom_1/(1.0-beta1**iter)
        term_2 = mom_2/(1.0-beta2**iter)
        update = eta*term_1/(np.sqrt(term_2)+delta)
        theta -= update
print("theta from ADAM using autograd")
print(theta)

our own inversion:
[[2.]
 [4.]
 [5.]]
theta from our own grad descent:
[[1.99679243]
 [4.00883833]
 [4.99576941]]
theta from our own grad descent, using adagrad:
[[4.25878806]
 [2.44746756]
 [3.42676722]]
theta from our own grad descent, using adagrad and momentum of 0.3:
[[3.84156197]
 [3.86583573]
 [4.1066359 ]]
theta from SGD with AdaGrad and without momentum
[[2.05027738]
 [3.87729596]
 [5.06184274]]
theta from SGD with AdaGrad and with momentum
[[1.99523586]
 [4.01246142]
 [4.99418317]]
theta from RMSprop using autograd
[[2.79829355]
 [2.37775333]
 [5.66509058]]
theta from ADAM using autograd
[[2.01684846]
 [4.00958899]
 [5.02237093]]
